In [1]:
import pandas as pd
import numpy as np

from bases import linear, polynomial_1d, polynomial, sinusoidal_1d, sinusoidal, radial
from metrics import MSE
from surrogate_model import SurrogateModel
from error_estimate import bootstrap_sets, bootstrap_estimate, leave_one_out_bootstrap_estimate, bootstrap_632_estimate

In [2]:
df = pd.read_csv('winequality-red.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
target = 'quality'

X = df[df.columns.drop(target)].to_numpy()
y = df[target].to_numpy()

m = X.shape[0]
n = X.shape[1]
a = np.array([min([X[j][i] for j in range(m)]) for i in range(n)])
b = np.array([max([X[j][i] for j in range(m)]) for i in range(n)])


In [7]:
bases_sets = {
    'linear'                              : linear(n),
    'polynomial (k=2)'                    : polynomial(n, 2),
    'sinusoidal (k=2)'                    : sinusoidal(n, 2, a, b),
    'radial (f(r) = r, C = [(0.0, 0.0)])' : radial(lambda r: r, [np.zeros(n)])
}

smoothing = 0.02
tt_sets = bootstrap_sets(m, 5)

print('0.632 bootstrap estimation of generalization error on the original dataset:')
for name, bases in bases_sets.items():
    print(f' + {name}: {bootstrap_632_estimate(X, y, bases, smoothing, MSE, tt_sets)}')


0.632 bootstrap estimation of generalization error on the original dataset:
 + linear: 0.39571569998036127
 + polynomial (k=2): 0.39251829526394894
 + sinusoidal (k=2): 14.40899651937578
 + radial (f(r) = r, C = [(0.0, 0.0)]): 9.413440818221716
